In [4]:
# Import Pandas
import io
import pyperclip
import pandas as pd
from contextlib import redirect_stdout

In [5]:
# Load data and drop missing values
df = pd.read_excel('/home/cmcouto-silva/Insync/cmcouto.silva@usp.br/cmcouto-silva.github.io/static/archive/resume/datascience_curriculum.xlsx')

In [6]:
df.head()

,Name,Type,Link,Institution,Platform,Tags,Completed,Certificate,Completion_date,Additional_Info
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Statistics with R Specialization,Specialization,https://www.coursera.org/specializations/stati...,Duke University,Coursera,"R, Statistics",NaN,NaN,NaN,PLUS
2,Introduction to Probability and Data,Course,https://www.coursera.org/learn/probability-int...,Duke University,Coursera,"R, Statistics",False,NaN,NaN,PLUS
3,Inferential Statistics,Course,https://www.coursera.org/learn/inferential-sta...,Duke University,Coursera,"R, Statistics",False,NaN,NaN,PLUS
4,Linear Regression and Modeling,Course,https://www.coursera.org/learn/linear-regressi...,Duke University,Coursera,"R, Statistics",False,NaN,NaN,PLUS


In [7]:
logi = df.isna().all(axis=1).shift(1, fill_value=False)

In [8]:
df.insert(0, 'Group',  df.loc[logi, 'Name'])

In [9]:
df.dropna(how='all', inplace=True)

In [10]:
df.loc[:, 'Group'] = df.Group.ffill()

In [11]:
# Fix dtypes
df['Completion_date'] = pd.to_datetime(df.Completion_date)
df['Completed'] = df.Completed.fillna(0).replace({'doing':0}).astype(bool)

In [12]:
# Get courses with certificate
df = df[df.Completed & df.Certificate.fillna(False).astype(bool)]
df = df[['Group', 'Name', 'Type', 'Link', 'Institution', 'Platform', 'Certificate', 'Completion_date']]

In [13]:
df.head()

,Group,Name,Type,Link,Institution,Platform,Certificate,Completion_date
16,Mastering Software Development in R Specializa...,The R Programming Environment,Course,https://www.coursera.org/learn/r-programming-e...,Johns Hopkins University,Coursera,https://www.coursera.org/account/accomplishmen...,2018-03-08
61,Python for Everybody Specialization,Programming for Everybody (Getting Started wit...,Course,https://www.coursera.org/learn/python?speciali...,University of Michingan,Coursera,https://www.coursera.org/account/accomplishmen...,2020-06-20
62,Python for Everybody Specialization,Python Data Structures,Course,https://www.coursera.org/learn/python-data?spe...,University of Michingan,Coursera,https://www.coursera.org/account/accomplishmen...,2020-06-20
68,Python 3 Programming Specialization,Python Basics,Course,https://www.coursera.org/learn/python-basics?s...,University of Michingan,Coursera,https://www.coursera.org/account/accomplishmen...,2020-03-18
69,Python 3 Programming Specialization,"Python Functions, Files, and Dictionaries",Course,https://www.coursera.org/learn/python-function...,University of Michingan,Coursera,https://www.coursera.org/account/accomplishmen...,2020-04-03


In [14]:
df.Group.value_counts()

DataCamp Career Track: Data Analyst with Python       14
DataCamp Career Track: Data Scientist with Python     13
DataCamp Career Track: Python Programmer               6
IBM Data Science Professional Certificate              5
Kaggle Minicourses                                     5
DataCamp - Introductory courses                        5
Applied Data Science Specialization                    3
Python for Everybody Specialization                    2
Python 3 Programming Specialization                    2
Mastering Software Development in R Specialization     1
Statistics with Python Specialization                  1
LinkedIn Learning                                      1
Name: Group, dtype: int64

In [15]:
featured_groups = [
    'IBM Data Science Professional Certificate',
    'DataCamp Career Track: Data Analyst with Python'
]

In [23]:
df_feat = df[df.Group.isin(featured_groups)].copy()
df_feat.loc[:, 'Group'] = pd.Categorical(df_feat.Group, featured_groups)
df_feat = df_feat.reset_index().sort_values(['Group', 'index'], ascending=True).drop('index', axis=1)

In [24]:
df_plat = df[~df.Group.isin(featured_groups)]
df_plat.Platform.unique()

array(['Coursera', 'Kaggle', 'DataCamp', 'LinkedIn'], dtype=object)

In [25]:
df_plat['Platform'] = pd.Categorical(df_plat.Platform, ['DataCamp', 'Coursera', 'Kaggle', 'LinkedIn'])
df_feat = df_feat.reset_index().sort_values(['Group', 'index'], ascending=True).drop('index', axis=1)

In [26]:
featured_courses = df_feat[['Institution', 'Name']]
df_plat = (
    df_plat.merge(featured_courses, 'left', indicator=True)
    .query('_merge=="left_only"').drop('_merge', axis=1)
    .sort_values(['Platform', 'Completion_date'], ascending=False)
)

### Create tables with Markdown syntax

In [27]:
# Featured course groups
feat_md = io.StringIO()
with redirect_stdout(feat_md):
    for group, df in df_feat.groupby('Group'):
        print(f'#### {group}', '---', sep='\n')
        header = ('|Institution|Subject|Certificate|Date|', '|---|---|---|---|')
        print('\n'.join(header))
        df.apply(lambda row: print(
            f'|{row.Institution}'
            f'|<a href="{row.Link}" target="_blank" style="color:inherit">{row.Name}</a>',
            f'|<a href="{row.Certificate}" target="_blank">see certificate</a>',
            f'|{row.Completion_date.strftime("%b %Y")}|',
            sep=''
        ), axis=1)
        print()
    
feat_md = feat_md.getvalue()
# print(feat_md)

In [30]:
# All courses
plat_md = io.StringIO()
with redirect_stdout(plat_md):
    for group, df in df_plat.groupby('Platform'):
        df = df.sort_values('Completion_date', ascending=False)
        print(f'#### {group}', '---', sep='\n')
        header = ('|Institution|Subject|Certificate|Date|', '|---|---|---|---|')
        print('\n'.join(header))
        df.apply(lambda row: print(
            f'|{row.Institution}'
            f'|<a href="{row.Link}" target="_blank" style="color:inherit">{row.Name}</a>',
            f'|<a href="{row.Certificate}" target="_blank">see certificate</a>',
            f'|{row.Completion_date.strftime("%b %Y")}|',
            sep=''
        ), axis=1)
        print()
    
plat_md = plat_md.getvalue()
# print(plat_md)

### Create tables with HTML5 syntax

In [34]:
# Featured course groups
feat_html = io.StringIO()
with redirect_stdout(feat_html):
    for group, df in df_feat.groupby('Group'):

        print(f'<h4>{group}</h4><hr>')

        header = [
        '<table>', '<thead>', '<tr>',
        '<th class="institution">Institution</th>',
        '<th class="subject">Subject</th>',
        '<th class="certificate">Certificate</th>',
        '<th class="date">Date</th>',
        '</tr>', '</thead>', '<tbody>'
        ]

        [print(h) for h in header]

        df.apply(lambda row: print(
            f'<tr>',
            f'<td>{row.Institution}</td>',
            f'<td><a href="{row.Link}" target="_blank" style="color:inherit">{row.Name}</a></td>',
            f'<td class="certificate"><a href="{row.Certificate}" target="_blank">see certificate</a></td>',
            f'<td class="date">{row.Completion_date.strftime("%b %Y")}</td>',
            '</tr>',
            sep=''
        ), axis=1)

        print('</tbody>')
        print('</table>')
        print()
    
feat_html = feat_html.getvalue()
# print(feat_html)

In [36]:
# All courses
plat_html = io.StringIO()
with redirect_stdout(plat_html):
    for group, df in df_plat.groupby('Platform'):

        print(f'<h4>{group}</h4><hr>')

        header = [
        '<table>', '<thead>', '<tr>',
        '<th class="institution">Institution</th>',
        '<th class="subject">Subject</th>',
        '<th class="certificate">Certificate</th>',
        '<th class="date">Date</th>',
        '</tr>', '</thead>', '<tbody>'
        ]

        [print(h) for h in header]

        df.apply(lambda row: print(
            f'<tr>',
            f'<td>{row.Institution}</td>',
            f'<td><a href="{row.Link}" target="_blank" style="color:inherit">{row.Name}</a></td>',
            f'<td class="certificate"><a href="{row.Certificate}" target="_blank">see certificate</a></td>',
            f'<td class="date">{row.Completion_date.strftime("%b %Y")}</td>',
            '</tr>',
            sep=''
        ), axis=1)

        print('</tbody>')
        print('</table>')
        print()
    
plat_html = plat_html.getvalue()
# print(plat_html)

In [37]:
pyperclip.copy(feat_html)

In [38]:
pyperclip.copy(plat_html)